In [ ]:
import pandas as pd
df=pd.read_csv("/content/train.csv")
# df=df.head(50)
df.columns

Index(['essay_id', 'full_text', 'score'], dtype='object')

In [ ]:
from langchain_core.documents import Document

docs = []
for _, row in df.iterrows():
    docs.append(
        Document(
            page_content=row["full_text"],
            metadata={"essay_id": row["essay_id"], "essay_score": row["score"]}
        )
    )


In [ ]:

import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

In [ ]:
# !pip install -qU  langchain-google-genai

In [ ]:
# !pip install -U langchain langchain-core langchain-community langchain-google-genai


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query(docs[0].page_content)
vector[:5]

[0.025306520983576775,
 -0.008368444629013538,
 0.04489908739924431,
 -0.03559477999806404,
 0.021746693179011345]

In [ ]:
# !pip install -U chromadb langchain-chroma


In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./essay_db"
)


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "How can technology improve education?"
results = retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"--- Essay {i+1} ---")
    print(doc.metadata)
    print(doc.page_content[:200], "...")


In [ ]:
vectorstore = Chroma(
    persist_directory="./essay_db",
    embedding_function=embeddings
)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    # System message: sets context and rules
    ("system",
     """You are an expert writing tutor helping students improve their essays.
Your goal is to analyze the student's essay, compare it with high-scoring examples,
and give detailed, actionable feedback based on writing principles and the 5W–1H rule.
Use an encouraging and professional tone."""),

    # User message: contains essay + retrieved essays + instructions
    ("user",
     """The user has written the following essay:

[USER ESSAY]
{user_essay}

We retrieved some top-scoring essays on similar topics:

[REFERENCE ESSAYS]
{retrieved_essays}

Your task:
1. Compare the user's essay with the reference essays.
2. Identify areas of improvement in:
   - Structure & organization
   - Coherence & logical flow
   - Vocabulary & word choice
   - Clarity & conciseness
   - Grammar & syntax
   - Argumentation & reasoning
   - Engagement & tone

3. Evaluate if the essay answers the **5W–1H** aspects of good writing:
   - What: Is the topic clearly introduced?
   - Why: Does the essay explain its significance or purpose?
   - Where: Is the context or setting established?
   - When: Are time aspects clear (if relevant)?
   - Who: Are people or entities properly identified?
   - How: Is the process, reasoning, or impact explained?

4. Give specific, actionable feedback with examples where needed.
5. End with an improved version of the essay, keeping the user's tone and intent.

Respond strictly in this format and jut only this and nothing else other than the following:
Also make sure to answer in simple humanlike wordings and not complex words .

**Feedback Summary:**
- ...

**Detailed Suggestions (based on Writing & 5W–1H):**
1. ...
2. ...

**Refined Essay:**
...
""")
])


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
import random
parser = StrOutputParser()

def format_docs(docs):
    return "\n\n---\n\n".join([doc.page_content for doc in docs])

def retrieve_better(user_essay):
    predicted =random.randint(1,5)
    print(f"Predicted score (dummy): {predicted}")

    dynamic_retriever = vectorstore.as_retriever(
        search_kwargs={
            "k": 3,
            "filter": {"score": {"$gt": predicted}}
        }
    )
    docs = dynamic_retriever.invoke(user_essay)
    return format_docs(docs)


essay_chain = (
    {
        "user_essay": lambda x: x["user_essay"],
        "retrieved_essays": lambda x: retrieve_better(x["user_essay"])
    }
    | prompt
    | llm
    | parser
)

In [ ]:
user_input = {
    "user_essay": """
WHEN IS A PLAINTIFF ENTITLED TO RECOVER? A. A plaintiff who was injured as as result of some negligent conduct on the part of a defendant is entitled to recover compensation for such injury from that defendant.A plaintiff is entitled to a verdict if jury finds1. That a defendant was negligent, and2. That such negligence was a cause of injury to the plaintiff. Q. WHAT IS NEGLIGENCE? Negligence is the doing of something which a reasonably prudent person would not do, or the failure to do something
"""
}

result = essay_chain.invoke(user_input)
print(result)


In [ ]:
from langchain_core.output_parsers import StrOutputParser
import random
import distils

parser = StrOutputParser()
model_fol="./models"

def format_docs(docs):
    return "\n\n---\n\n".join([doc.page_content for doc in docs])

def retrieve_better(user_essay):
    predicted = random.randint(1,5)
    print(f"Predicted score (dummy): {predicted}")

    dynamic_retriever = vectorstore.as_retriever(
        search_kwargs={
            "k": 3,
            "filter": {"score": {"$gt": predicted}}
        }
    )
    docs = dynamic_retriever.invoke(user_essay)
    return format_docs(docs)


essay_chain = (
    {
        "user_essay": lambda x: x["user_essay"],
        "retrieved_essays": lambda x: retrieve_better(x["user_essay"])
    }
    | prompt
    | llm
    | parser
)